# Four point functions in the Potts and $O(n)$ models

This notebook reproduces some known results about Potts and $O(n)$ four-point functions, using the new Julia code.

In [4]:
Pkg.activate("..") # activate the parent environment
using BootstrapVirasoro
n_threads = Threads.nthreads()
if n_threads == 1
    println("You are using a single thread. Consider starting julia with more threads, for instance by setting
    the environment variable `export JULIA_NUM_THREADS=auto`")
end

  Activating project at `~/Documents/Recherche/projet_these/code/BootstrapVirasoro`


## Spectra

We create functions to generate the spectra of the $O(n)$ and Potts CFTs, with a cutoff:

\begin{align}
\mathcal{S}^{O(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in 2\mathbb{N}+1} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \frac12\mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
 \\
 \mathcal{S}^{PSU(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in\mathbb{N}^*} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
 \\
 \mathcal{S}^\text{Potts} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in\mathbb{N}^*} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \mathbb{N}+2\\ s\in\frac{1}{r}\mathbb{Z}}} \bigcup  \left\{ V_{P_{(0,s)}}\right\}_{s\in \mathbb{N}+\frac12}\ .
\end{align}


In [10]:
setprecision(BigFloat, 50, base=10)
c = CC(β=1/(big"0.8" + big"0.1"*im))

LoopFields = Dict(
    :On => vcat(
        [Field(c, r=r, s=s) for r in 1//2:1//2:50 for s in -10+1//r:1//r:10 if r*s%1 == 0],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:2:50]
    ),
    :PSUn => vcat(
        [Field(c, r=r, s=s) for r in 1:50 for s in -10+1//r:1//r:10],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:50]
    ),
    :Potts => vcat(
        [Field(c, r=r, s=s) for r in 2:50 for s in -10+1//r:1//r:10],
        [Field(c, r=0, s=s, diagonal=true) for s in 1//2:1:3//2],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:50]
    )
)

LoopSpectra = Dict(
    model => Spectrum(LoopFields[model], Δmax, interchiral=true)
    for model in (:On, :PSUn, :Potts)
);

LoopSpectrum(model, Δmax) = Spectrum(LoopFields[model], Δmax, interchiral=true);

## Bootstrap equations

We numerically solve

\begin{align}
\sum_{V \in \mathcal{S}^{(s)}} D^{(s)}_V \mathcal I^{(s)}_V (x) = \sum_{V \in \mathcal{S}^{(t)}} D^{(t)}_V \mathcal I^{(t)}_V (x) = \sum_{V \in \mathcal{S}^{(u)}} D^{(u)}_V \mathcal I^{(u)}_V (x),
\end{align}

for some channel spectra $\mathcal{S}^{(s)}, \mathcal{S}^{(t)}, \mathcal{S}^{(u)}$, where $\mathcal I$ are interchiral conformal blocks, and the $D$'s are four-point structure constants.

We solve this system as
\begin{equation}
\underbrace{
\begin{bmatrix}
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [-\mathcal I^{(t)}_{V_j}(x_i)]_{ij} & [0] \\
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [0] & [-\mathcal I^{(u)}_{V_j}(x_i)]_{ij}
\end{bmatrix}}_A
\begin{bmatrix}
[D^{(s)}_{V_j}]_j \\
[D^{(t)}_{V_j}]_j \\
[D^{(u)}_{V_j}]_j
\end{bmatrix} = 
\begin{bmatrix}
\sum_{V_j \in \text{ known}} D^{(t)}_{V_j} \mathcal{I}^{(t)}_{V_j}(1-x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)\\
\sum_{V_j \in \text{ known}} D^{(u)}_{V_j} \mathcal{I}^{(u)}_{V_j}(1/x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)
\end{bmatrix}
\end{equation}
where the $x_i$ take more values than there are unknowns, i.e. $A$ is a tall rectangular matrix. 

To check numerical convergence, we solve two subsystems and compare the solutions. If the solutions are close, we know the computation has converged.

## $\langle (\frac{1}{2}, 0)^2 (1, 0)^2 \rangle$

In [17]:
indices = ((1 // 2, 0), (1 // 2, 0), (1, 0), (1, 0))

function solve(indices; signature = Dict(:s => 0, :t => 0, :u => 0), diags = Dict(), Δmax=30)
    fields = [Field(c, r=r, s=s) for (r, s) in indices]
    co = Correlation(fields..., Δmax=Δmax)
    SOn = LoopSpectrum(:On, Δmax)

    Schan_On = ChannelSpectra(
        co, SOn,
        signature=signature
    )

    for (chan, VP) in diags
        add!(Schan_On, VP)
    end

    b = BootstrapSystem(Schan_On)
    compute_linear_system!(b)
    solve!(b, precision_factor=1)
    return b.consts
end

@time solve(indices, signature=Dict(:s => 1, :t => 1//2, :u => 3//2))

 18.779523 seconds (204.24 M allocations: 8.884 GiB, 11.99% gc time, 2.48% compilation time)


┌ Info: system size: (118, 112)
└ @ BootstrapVirasoro /Users/paulroux/Documents/Recherche/projet_these/code/BootstrapVirasoro/src/bootstrap_equations.jl:159


Channel s
Fields         | Structure constants          | Errors                     
--------------------------------------------------------------------
V_{(1, 0)}     |  1.00000e+00 + 0.00000e+00im | 0.00000e+00 + 0.00000e+00im
V_{(1, 1)}     | -1.00000e+00 - 3.40453e-42im |  7.25656e-42 + 0.00000e+00im
V_{(2, -1//2)} |  9.23413e-43 - 1.90505e-42im |  1.87617e+00 + 0.00000e+00im
V_{(2, 0)}     |  7.86010e-44 + 7.79067e-44im |  2.19715e+00 + 0.00000e+00im
V_{(2, 1//2)}  |  1.40967e-43 + 2.65604e-44im |  2.08496e+00 + 0.00000e+00im
V_{(2, 1)}     |  4.27761e-44 + 3.35279e-44im |  1.60787e+00 + 0.00000e+00im
V_{(3, -2//3)} |  1.38446e-44 + 4.26249e-45im |  6.00548e+00 + 0.00000e+00im
V_{(3, -1//3)} |  8.34521e-42 + 9.21866e-44im |  1.26707e+01 + 0.00000e+00im
V_{(3, 0)}     |  1.44290e-42 - 7.75976e-42im |  6.88206e+00 + 0.00000e+00im
V_{(3, 1//3)}  | -6.92051e-42 - 5.55178e-42im |  3.93346e+00 + 0.00000e+00im
V_{(3, 2//3)}  | -7.94270e-42 + 8.56296e-42im |  4.81045e+00 + 0.00000e+00im

In [ ]:
b.matrix.LHS